# Method **1**

**Siamese model on benchmark dataset**

In [1]:

import tensorflow as tf
print(tf.__version__)

import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense,Input,Embedding,Bidirectional,LSTM,Lambda,Reshape
from tensorflow.keras.models import Model

2.4.1


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# Date : 24.01.2021


df=pd.read_csv('/content/drive/MyDrive/Data/quora duplicate identification/train.csv',header=0)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
#some exploration

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [5]:
len(df['qid1'].unique()),len(df['qid2'].unique())

(290654, 299364)

In [6]:
print(f'No. of questions common in both ids {len(np.intersect1d(df.qid1.values,df.qid2.values))}')

No. of questions common in both ids 52085


In [7]:
print(f'Therefore total no. of unique questions are {len(df.qid1.unique())+len(df.qid2.unique())-len(np.intersect1d(df.qid1.values,df.qid2.values))}')



Therefore total no. of unique questions are 537933


In [8]:
df=df.dropna()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404287 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404287 non-null  int64 
 1   qid1          404287 non-null  int64 
 2   qid2          404287 non-null  int64 
 3   question1     404287 non-null  object
 4   question2     404287 non-null  object
 5   is_duplicate  404287 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 21.6+ MB


In [10]:
#basic text cleanising

x1=df['question1'].values
x2=df['question2'].values

In [11]:
x1[:10]

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       'Why am I mentally very lonely? How can I solve it?',
       'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
       'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
       'Should I buy tiago?', 'How can I be a good geologist?',
       'When do you use シ instead of し?',
       'Motorola (company): Can I hack my Charter Motorolla DCX3400?'],
      dtype=object)

In [12]:
x1,x2=[i.lower() for i in x1],[i.lower() for i in x2]

In [13]:
x1[:10]

['what is the step by step guide to invest in share market in india?',
 'what is the story of kohinoor (koh-i-noor) diamond?',
 'how can i increase the speed of my internet connection while using a vpn?',
 'why am i mentally very lonely? how can i solve it?',
 'which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
 'astrology: i am a capricorn sun cap moon and cap rising...what does that say about me?',
 'should i buy tiago?',
 'how can i be a good geologist?',
 'when do you use シ instead of し?',
 'motorola (company): can i hack my charter motorolla dcx3400?']

In [14]:
# some regex based cleaning
import re

def regex_clean(text):
  text=re.sub(r'[^\w\s]', '',text)               # texts without punctuation  \ used just in case if alpha-numeric characters has to be replaced for beteer replacements
  text=re.sub(r'[^A-Za-z0-9]+', ' ', text)       # texts contains only alpha-numeric characters
  text=re.sub('  ', '', text)                    # removing an extra spaces
  text=re.sub('   ','',text)                     # removing two extra spaces
  return text

In [15]:
#for normalized scores

!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [16]:
import spacy

nlp=spacy.load("en_core_web_sm")



In [17]:
#test
"""mystring='motorola (company): can i hack my charter motorolla dcx3400?'
re.sub(r'[^A-Za-z0-9]+', ' ', mystring)"""
 

"mystring='motorola (company): can i hack my charter motorolla dcx3400?'\nre.sub(r'[^A-Za-z0-9]+', ' ', mystring)"

Unnormalized data

In [18]:
x1_data=[regex_clean(text) for text in x1]
x2_data=[regex_clean(text) for text in x2]

In [19]:
y_data=tf.one_hot(df['is_duplicate'].values,2)

In [20]:
x1_data[:3]

['what is the step by step guide to invest in share market in india',
 'what is the story of kohinoor kohinoor diamond',
 'how can i increase the speed of my internet connection while using a vpn']

In [21]:
# contraction replacemets
!pip install contractions

In [22]:
#Testing the package in pypi

import contractions
"""contractions.fix("i'll call you")"""

'contractions.fix("i\'ll call you")'

In [23]:
x1_data=[contractions.fix(data) for data in x1_data]
x2_data=[contractions.fix(data) for data in x2_data]

In [24]:
# acronym identification and replacement

# this is impossible for this task but if there is a small dataset on which fine tuning can be done then it could be useful.

In [25]:
' '.join([str(i) for i in nlp("this is impossible for this task but if there is a small dataset on which fine tuning on this model can be done then it could be useful.")])

'this is impossible for this task but if there is a small dataset on which fine tuning on this model can be done then it could be useful .'

Normalized data

In [26]:
n_x1_data=[' '.join([str(token.lemma_) for token in nlp(text)]) for text in x1_data[:1000]]    #lematization and singularization
n_x2_data=[' '.join([str(token.lemma_) for token in nlp(text)]) for text in x2_data[:1000]]

In [27]:
#using tensorflow tokenizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [28]:
tokenizer1=Tokenizer()
tokenizer1.fit_on_texts(n_x1_data+n_x2_data)

In [29]:
n_sequences=tokenizer1.texts_to_sequences(n_x1_data)
n_sequences_2=tokenizer1.texts_to_sequences(n_x2_data)

In [30]:
n_x1_sequence_data=pad_sequences(n_sequences,maxlen=144)
n_x2_sequence_data=pad_sequences(n_sequences_2,maxlen=144)

In [31]:
n_vocab=tokenizer1.word_index

Un-normalized data prepn

In [32]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(x1_data+x2_data)

In [33]:
sequences=tokenizer.texts_to_sequences(x1_data)
sequences_2=tokenizer.texts_to_sequences(x2_data)

In [34]:
x1_sequence_data=pad_sequences(sequences,maxlen=144)
x2_sequence_data=pad_sequences(sequences_2,maxlen=144)

In [35]:
x1_data[:10]

['what is the step by step guide to invest in share market in india',
 'what is the story of kohinoor kohinoor diamond',
 'how can i increase the speed of my internet connection while using a vpn',
 'why am i mentally very lonely how can i solve it',
 'which one dissolve in water quikly sugar salt methane and carbon di oxide',
 'astrology i am a capricorn sun cap moon and cap risingwhat does that say about me',
 'should i buy tiago',
 'how can i be a good geologist',
 'when do you use instead of ',
 'motorola company can i hack my charter motorolla dcx3400']

In [36]:
tokenizer.word_index['capricorn']

9959

In [37]:
vocab=tokenizer.word_index

In [38]:
# #lets try google embeddings according to the paper
# !mkdir ~/.kaggle
# !cp /content/drive/MyDrive/kaggle.json /root/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle datasets download alvations/vegetables-google-word2vec

In [39]:
# !unzip /content/vegetables-google-word2vec.zip

In [40]:
embeddings=np.load('/content/word2vec.news.negative-sample.300d.npy')
embeddings

array([[ 1.1291504e-03, -8.9645386e-04,  3.1852722e-04, ...,
        -1.5640259e-03, -1.2302399e-04, -8.6307526e-05],
       [ 7.0312500e-02,  8.6914062e-02,  8.7890625e-02, ...,
        -4.7607422e-02,  1.4465332e-02, -6.2500000e-02],
       [-1.1779785e-02, -4.7363281e-02,  4.4677734e-02, ...,
         7.1289062e-02, -3.4912109e-02,  2.4169922e-02],
       ...,
       [-1.9653320e-02, -9.0820312e-02, -1.9409180e-02, ...,
        -1.6357422e-02, -1.3427734e-02,  4.6630859e-02],
       [ 3.2714844e-02, -3.2226562e-02,  3.6132812e-02, ...,
        -8.8500977e-03,  2.6977539e-02,  1.9042969e-02],
       [ 4.5166016e-02, -4.5166016e-02, -3.9367676e-03, ...,
         7.9589844e-02,  7.2265625e-02,  1.3000488e-02]], dtype=float32)

In [41]:
tokens = [line.strip() for line in open('/content/word2vec.news.negative-sample.300d.txt')]
embeddings[tokens.index('mobile')]

array([ 0.06494141, -0.328125  , -0.20605469, -0.15234375, -0.27734375,
        0.09912109,  0.10205078, -0.06054688,  0.10009766, -0.01965332,
        0.04516602, -0.0088501 , -0.06542969, -0.10546875,  0.07568359,
       -0.18066406,  0.07617188, -0.05249023, -0.18652344,  0.12890625,
        0.14941406, -0.10253906,  0.03295898,  0.08691406, -0.03588867,
       -0.11376953, -0.1328125 ,  0.26953125,  0.265625  , -0.09960938,
       -0.17773438,  0.0246582 , -0.02001953, -0.07568359, -0.05175781,
       -0.14257812, -0.07177734,  0.02563477,  0.08056641,  0.17773438,
       -0.16992188, -0.12451172,  0.06640625,  0.12988281, -0.02697754,
       -0.09472656, -0.12255859,  0.0534668 , -0.03442383, -0.10058594,
        0.11962891,  0.04150391,  0.01226807, -0.06738281, -0.01867676,
        0.18261719, -0.15039062,  0.09179688,  0.14257812,  0.15039062,
       -0.11035156, -0.01031494, -0.18164062,  0.10449219, -0.19921875,
        0.19726562, -0.10253906,  0.25195312,  0.00267029, -0.00

In [42]:
len(vocab)

108024

In [43]:
# embeddings_matrix = np.zeros((len(vocab)+1, 300));
# for word, i in vocab.items():
#   if word in tokens:
#     embedding_vector = embeddings[tokens.index(word)];
#     if embedding_vector is not None:
#         embeddings_matrix[i] = embedding_vector;
#   else:
#     embeddings_matrix[i]=np.array([0]*300)

In [44]:
import pickle

"""with open('embeddings.pkl','wb') as files:
  pickle.dump(embeddings_matrix,files)"""

"with open('embeddings.pkl','wb') as files:\n  pickle.dump(embeddings_matrix,files)"

In [45]:
with open('/content/drive/MyDrive/ML Model weights/quora/embeddings.pkl','rb') as files:
  embeddings=pickle.load(files)

In [46]:
embeddings.shape

(108025, 300)

In [47]:
#model(siamese network) using functional API

"""embedding_layer=Embedding(vocab_len+1,100,input_length=x_train_data_1.shape[-1])
lstm=LSTM(512,dropout=0.3)

s1=Input(shape=(x_train_data_1.shape[-1]))
embed=embedding_layer(s1)
x1=lstm(embed)

s2=Input(shape=(x_train_data_1.shape[-1]))
embed_1=embedding_layer(s2)
y1=lstm(embed_1)

#eucledian distance is not used here.i tried with simple DNN
com=concatenate([x1,y1])                    
com=Dropout(0.2)(com)
com=BatchNormalization()(com)
com=Dense(256,activation='relu')(com)
com=Dropout(0.2)(com)
pred=Dense(2,activation='softmax')(com)        

model=Model([s1,s2],pred)

model.compile(optimizer=Adam(0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()"""

"embedding_layer=Embedding(vocab_len+1,100,input_length=x_train_data_1.shape[-1])\nlstm=LSTM(512,dropout=0.3)\n\ns1=Input(shape=(x_train_data_1.shape[-1]))\nembed=embedding_layer(s1)\nx1=lstm(embed)\n\ns2=Input(shape=(x_train_data_1.shape[-1]))\nembed_1=embedding_layer(s2)\ny1=lstm(embed_1)\n\n#eucledian distance is not used here.i tried with simple DNN\ncom=concatenate([x1,y1])                    \ncom=Dropout(0.2)(com)\ncom=BatchNormalization()(com)\ncom=Dense(256,activation='relu')(com)\ncom=Dropout(0.2)(com)\npred=Dense(2,activation='softmax')(com)        \n\nmodel=Model([s1,s2],pred)\n\nmodel.compile(optimizer=Adam(0.0001),loss='categorical_crossentropy',metrics=['accuracy'])\nmodel.summary()"

In [48]:
# siamese model development 

class Model(tf.keras.Model):
  """for questions column"""
  
  def __init__(self,units_1,units_2,vocab,name):
    super().__init__(name=f'input_network {name}')
    self.input_1=tf.keras.layers.Input(shape=(units_1,),name=name)
    self.embedding=tf.keras.layers.Embedding(len(vocab)+1,300,weights=[embeddings],trainable=False)
    self.lstm=tf.keras.layers.LSTM(units_2, name=f'LSTM {name}')
    

  def call(self,inputs):
    x=self.embedding(inputs)
    x=self.lstm(x)


    return x


In [49]:
class SiameseNetwork(tf.keras.Model):

  def __init__(self,unit1,unit2,vocab):
    super(SiameseNetwork,self).__init__()
    self.dropout1=tf.keras.layers.Dropout(0.2,name='dropout1')
    self.dropout2=tf.keras.layers.Dropout(0.2,name='dropout2')
    self.f_dense=tf.keras.layers.Dense(2,activation='softmax' ,name='final_dense')
    self.layer1=Model(unit1,unit2,vocab,'input1')
    self.layer2=Model(unit1,unit2,vocab,'input2')

  def call(self,input1,input2,training):
    x1=self.layer1(input1)
    x2=self.layer2(input2)
    x=tf.keras.layers.concatenate([x1,x2])
    x=self.dropout1(x)
    x=self.dropout2(x)
    x=self.f_dense(x)
    return x

In [50]:
epochs=50

In [51]:
loss_fn=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

In [54]:
#Custom Training 

@tf.function
def train(x1_train_data,x2_train_data,y_train_data,vocab,training=True):
    # model1=Model(144,75,vocab)
    # # x1=tf.keras.layers.Input(shape=(144,))
    # # Initialized_m1= model1(x1,training=False) # calling build method
    # model2=Model(144,75,vocab)
    # # Initialized_m2= model2(x1,training=False) # calling build method
    f_model=SiameseNetwork(144,75,vocab)
    batches=16
    for epoch in range(epochs):
      
      print(f"epoch number {epoch}")
      dl=int(len(x1_train_data)/batches)
      length=0
      for batch in range(batches):
        with tf.GradientTape() as tape:
          # output_logits_1=model1(x1_train_data[length:dl+length],training=training)
          # output_logits_2=model2(x2_train_data[length:dl+length],training=training)
          final_logits=f_model(x1_train_data[length:dl+length],x2_train_data[length:dl+length],training=training)
          loss_val=loss_fn(y_train_data[length:dl+length],final_logits)

        grads=tape.gradient(loss_val,f_model.trainable_weights)
        optimizer.apply_gradients(zip(grads,f_model.trainable_weights))
        
        train_acc_metric.update_state(y_train_data[length:dl+length], final_logits)
        length+=dl
        

        print(f'loss value after {batch}th batch : {loss_val}\n')
        

      train_acc = train_acc_metric.result()
      print('\n')
      print(f'Training acc over epoch {epoch} : {train_acc}\n\n') 
      train_acc_metric.reset_states()

    return f_model








In [52]:
tf.config.run_functions_eagerly(True)

In [55]:
f_model=train(x1_sequence_data[:10000],x2_sequence_data[:10000],y_data[:10000],vocab,training=True) #just 10k data taken for testing



epoch number 0
loss value after 0th batch : 0.682925820350647

loss value after 1th batch : 0.677506685256958

loss value after 2th batch : 0.6725709438323975

loss value after 3th batch : 0.68758624792099

loss value after 4th batch : 0.6715638041496277

loss value after 5th batch : 0.6712026596069336

loss value after 6th batch : 0.672799289226532

loss value after 7th batch : 0.682036817073822

loss value after 8th batch : 0.6777960658073425

loss value after 9th batch : 0.6643145680427551

loss value after 10th batch : 0.6565314531326294

loss value after 11th batch : 0.656547486782074

loss value after 12th batch : 0.6610345244407654

loss value after 13th batch : 0.6556926369667053

loss value after 14th batch : 0.6622159481048584

loss value after 15th batch : 0.6770159602165222



Training acc over epoch 0 : 0.6182000041007996


epoch number 1
loss value after 0th batch : 0.6695804595947266

loss value after 1th batch : 0.6607037782669067

loss value after 2th batch : 0.6508120

In [55]:
f_model

In [56]:
f_model.summary()

Model: "siamese_network"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout1 (Dropout)           multiple                  0         
_________________________________________________________________
dropout2 (Dropout)           multiple                  0         
_________________________________________________________________
final_dense (Dense)          multiple                  302       
_________________________________________________________________
input_network input1 (Model) multiple                  32520300  
_________________________________________________________________
input_network input2 (Model) multiple                  32520300  
Total params: 65,040,902
Trainable params: 225,902
Non-trainable params: 64,815,000
_________________________________________________________________


In [ ]:
type(f_model)

__main__.SiameseNetwork

In [59]:
y_pred=tf.argmax(f_model(x1_sequence_data[:1000],x2_sequence_data[:1000]),-1)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true.numpy(),y_pred.numpy())

0.72

Due to excessive memory usage couldnt train this model to give its best performance

In [64]:
f_model.load_weights('QQP.h5')